# Python Dictionary Hash Lookup: From Keys to Lightning Speed

## Why is looking for a specific element in a dict so different than looking in a list?


In [3]:
N = 100_000_000
mylist = [i for i in range(N)]
mydict = {i:1 for i in range(N)}

import random
rand_numb = random.randint(0,N)
print(rand_numb)

91745967


In [4]:
# Capture the timing results
dict_result = %timeit -o rand_numb in mydict
list_result = %timeit -o rand_numb in mylist

# Extract the best times
dict_time = dict_result.best
list_time = list_result.best

# Calculate and display the difference
speedup = list_time / dict_time
print(f"Dictionary is {speedup:,.0f}x faster!")

36.5 ns ± 0.521 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
596 ms ± 6.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Dictionary is 16,469,160x faster!


50

### Now, let's try to find the intersection of two ensembles.

In [23]:
# Example 2: Intersection of two collections
import random

# Create two lists with 10,000 random elements each from range 100,000
N_items = 10_000
N_range = 100_000

# Generate unique random samples
list1 = random.sample(range(N_range), N_items)  # No duplicates
list2 = random.sample(range(N_range), N_items)  # No duplicates
# Convert to sets, sets are similar to dictionary but without values
set1 = set(list1)
set2 = set(list2)

print(f"Finding intersection of two collections with {N_items:,} items each")
print(f"Items drawn from range 0 to {N_range:,}")

# Method 1: List intersection (naive approach)
def list_intersection(l1, l2):
    results = []
    for item in l1: 
        if item in l2:
            results.append(item)
    return results

# Method 2: Set intersection (optimized)
def set_intersection(s1, s2):
    return s1 & s2

print("List intersection (O(n²) - nested loops):")
list_result = %timeit -o list_intersection(list1, list2)

print("Set intersection (O(n) - hash table magic):")
set_result = %timeit -o set_intersection(set1, set2)

# Calculate speedup
list_time = list_result.best
set_time = set_result.best
speedup = list_time / set_time

Finding intersection of two collections with 10,000 items each
Items drawn from range 0 to 100,000
List intersection (O(n²) - nested loops):
709 ms ± 1.79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Set intersection (O(n) - hash table magic):
162 µs ± 992 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
print(f"List intersection: {list_time*1000:.1f} milliseconds")
print(f"Set intersection: {set_time*1000:.1f} milliseconds")
print(f"Sets are {speedup:,.0f}x faster!")
print("="*50)

# Verify they give same results (just different order)
list_result_actual = list_intersection(list1, list2)
set_result_actual = set_intersection(set1, set2)
print(f"Found {len(list_result_actual)} common items (list method)")
print(f"Found {len(set_result_actual)} common items (set method)")

List intersection: 706.9 milliseconds
Set intersection: 0.2 milliseconds
Sets are 4,398x faster!
Found 988 common items (list method)
Found 988 common items (set method)


## Why is it so much faster with Dictionaries or Sets
## Answer: the hash map
## But what is hashing ? 
Hashing is a mathematical function that takes any input (like a string or number) and converts it into a fixed-size integer that can be used as a direct address to instantly locate data in memory.

- It's deterministic, meaning you always get the same results.
- It tries to spread the results as much as possible (ideally, two inputs don't have the same hashed outputs)
- There are many hash functions

Basic polynomial hash:
```python
hash = 0
for each character c in string:
    hash = (hash * 31 + ascii_value(c)) % table_size

hash = 0
hash = (0 * 31 + 99) % 8 = 99 % 8 = 3    # 'c' = 99
hash = (3 * 31 + 97) % 8 = 190 % 8 = 6   # 'a' = 97  
hash = (6 * 31 + 116) % 8 = 302 % 8 = 6  # 't' = 116

"cat" → position 6
```

In [25]:
print(hash("apple"))
print(hash("banana"))  
print(hash(42))  
print(hash([1,2,3])) # can't hash a list

2252410359971589713
-3206993190246825253
42


TypeError: unhashable type: 'list'

## Internal Python logic (simplified)

```python
hash_value = hash("apple")      # → 2252410359971589713
array_size = 16                 # Current internal array size
position = hash_value % array_size  # → 1847203847 % 16 = 7
```

- Python looks in array[7] for "apple"
- Position 7 maps to a memory address
- CPUs can access any memory address almost instantly through what's called **random access**.

CPU: "Give me the data at address 0x1000038"  
Memory Controller: *decodes address* → Row 1000, Column 38 of RAM chip #3  
RAM: *sends back 8 bytes of data*  
CPU: "Got it!" (stores in register) 


Random access: CPU calculates exact address → jumps directly there  
Sequential access: CPU starts at beginning → reads address 0, then 1, then 2... until found  


RAM access: \~50\-100 nanoseconds   
But with CPU caches, often much faster (\~1\-10 nanoseconds)  

## What does this mean for looking up a value in a dict?
When you create a dictionary in Python, **Python's hash table implementation** will compute the hash and take the modulo against the size of the internal array for each key. It will then store the **key-value pair** at this array position.

So, when you want to retrieve a value by its key, Python computes `hash(key) % array_size` to find the array position, then looks at that position in memory.

## What happens when searching in a list?
It's different for a list - we have to work sequentially. **When searching for a specific value** (like `value in my_list`), Python must examine each element one by one until it either finds the target value or reaches the end of the list.


## Slide 16: What Happens When hash() % array_size Collides?
**When Different Keys Map to the Same Position**

```python
hash("apple") % 8   # → 3
hash("grape") % 8   # → 3  # Collision!
```

**Python's solution: Open Addressing with Probing**
1. Check position 3 → occupied by "apple"
2. Follow predetermined sequence: try positions 6, 1, 4...  
3. Find empty slot at position 6 → store "grape" there
4. **Remember this path for future lookups!**

**When looking up "grape" later:**
- Calculate: hash("grape") % 8 = 3
- Check position 3 → find "apple" (wrong key)
- Follow same probing sequence → position 6 → find "grape"!





## What Happens When We Increase the Array Size?
**Dynamic Resizing: Keeping Performance Optimal**

**When dictionary gets ~66% full, Python doubles the array size:**

```python
# Before: array_size = 8
hash("apple") % 8  = 3
hash("grape") % 8  = 3  # Collision!

# After resize: array_size = 16  
hash("apple") % 16 = 11  # Different position!
hash("grape") % 16 = 3   # No more collision!
```

**The resizing process:**
1. Create new, larger array (typically double size)
2. Rehash every existing key with new array size
3. Move all items to their new positions
4. Delete old array

**Result: Fewer collisions, faster lookups, maintained O(1) performance**



## The Memory Trade-off
**Speed Comes at a Cost**

```python
import sys
print(f"Set with 1,000 items: {sys.getsizeof(set1):,} bytes")
print(f"List with 1,000 items: {sys.getsizeof(list1):,} bytes")
```
```
Set with 1,000 items: 524,504 bytes
List with 1,000 items: 80,056 bytes
```

**Why hash tables use more memory:**
- **Sparse arrays**: Keep ~33% empty slots to minimize collisions
- **Hash caching**: Store hash values to avoid recalculation  
- **Metadata overhead**: Bookkeeping for dynamic resizing
- **Result: ~6x more memory for same data**

**The fundamental trade-off: Memory for Speed**
- Hash tables: More memory, O(1) operations
- Lists: Less memory, O(n) search operations

## Why Can't We Hash List Elements for Faster Retrieval?
**"Why don't we just hash each element in a list to make lookups O(1)?"**

This seems like an obvious optimization - if hashing works so well for dictionaries, why not apply it to lists?


## Problem #1 - Not All Elements Are Hashable

```python
# These work fine in lists but CAN'T be hashed:
my_list = [
    [1, 2, 3],           # Nested list - mutable
    {"key": "value"},    # Dictionary - mutable  
    [[1, 2], [3, 4]],    # List of lists - mutable
    {1, 2, 3}            # Set - mutable
]

# Trying to hash them fails:
hash([1, 2, 3])        # TypeError: unhashable type: 'list'
hash({"key": "value"}) # TypeError: unhashable type: 'dict'
```

**Hash tables require immutable, hashable keys:**
- Lists can contain ANY Python object
- Hash tables can only use hashable objects as keys
- Many common list elements (nested lists, dicts, sets) can't be hashed


## Problem #2 - Lists Preserve Order and Position

```python
my_list = ["apple", "banana", "cherry", "apple"]
print(my_list[0])  # → "apple" (first occurrence)
print(my_list[3])  # → "apple" (second occurrence)
print(my_list.index("apple"))  # → 0 (finds FIRST occurrence)
```

**Hash tables destroy positional information:**
- Hash("apple") always maps to the same position
- But "apple" appears at both index 0 AND index 3 in the list
- A hash table can't tell you which occurrence you want


# Data Structures: Pros and Cons Summary

## Lists
**✅ Pros:**
- **Ordered**: Maintains insertion order and position
- **Duplicates allowed**: Can store the same value multiple times
- **Index access**: Direct access to any position - `my_list[5]` is O(1)
- **Flexible**: Can contain any Python object (mutable or immutable)
- **Rich operations**: append, insert, pop, slice, reverse, sort
- **Memory efficient**: Compact storage, only ~8 bytes per item reference

**❌ Cons:**
- **Slow search**: Finding an item is O(n) - must check each element
- **Slow membership testing**: `item in my_list` is O(n)
- **Slow operations on large lists**: remove(), index() are O(n)

**💡 Best for:** Sequences, ordered data, when you need positional access


## Sets
**✅ Pros:**
- **Lightning fast lookup**: `item in my_set` is O(1) 
- **Fast set operations**: union, intersection, difference all optimized
- **Automatic deduplication**: No duplicate values
- **Mathematical operations**: Natural for set theory operations

**❌ Cons:**
- **No order**: Items have no meaningful sequence (though Python 3.7+ preserves insertion order)
- **No duplicates**: Can't store the same value twice
- **No indexing**: Can't access "the 3rd item" - `my_set[2]` doesn't exist
- **Memory hungry**: ~6x more memory than equivalent list
- **Limited element types**: Only hashable (immutable) objects allowed

**💡 Best for:** Membership testing, eliminating duplicates, set operations

## Dictionaries
**✅ Pros:**
- **Ultra-fast key lookup**: `my_dict[key]` is O(1)
- **Key-value mapping**: Associate data with meaningful identifiers
- **Ordered**: Maintains insertion order (Python 3.7+)
- **Flexible values**: Values can be any Python object
- **Rich operations**: get(), keys(), values(), items(), update()

**❌ Cons:**
- **Keys must be hashable**: No lists, sets, or dicts as keys
- **No duplicate keys**: Each key can appear only once
- **Memory overhead**: Significant memory usage for hash table structure
- **No positional access**: Can't get "the 3rd item" without iteration
- **Two-part structure**: More complex than simple sequences

**💡 Best for:** Key-value relationships, fast lookup by identifier, caching

## The Bottom Line: Choose Your Tool

**When to use each:**

🔹 **List** → You need order, positions, or duplicates  
🔹 **Set** → You need fast "does this exist?" checks  
🔹 **Dict** → You need fast "what's associated with this key?" lookups

**Memory vs Speed Trade-off:**
- **Lists/Tuples**: Minimal memory, slower search
- **Sets/Dicts**: More memory, lightning-fast access

**The Golden Rule:** Pick the data structure that matches your primary access pattern. Don't try to force one structure to do everything!
